<a href="https://colab.research.google.com/github/ealatorr/sds510/blob/main/Module_4/mod_4_basics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Module 4 Basics

Firstly I am importing the necessary packages.

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

Establishing keywords

In [ ]:
keywords = ["medicine", "breakthrough", "upcoming"]

Now Im setting up the function to grap a single article and return it as a long string.

In [ ]:
def grabarticle(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(" ", strip=True)
    return text

Next creating the CSV file.

In [ ]:
csv_name = "mod4_results.csv"
try:
    df = pd.read_csv(csv_name)
    # Make sure only url + keyword columns exist
    df = df[["url", "keyword"]]
    print("Loaded existing CSV:", csv_name)
except Exception:
    df = pd.DataFrame(columns=["url", "keyword"])
    print("starting new csv:", csv_name)
df.head()


Loaded existing CSV: mod4_results.csv


,url,keyword
0,https://news.asu.edu/20251125-health-and-medic...,medicine
1,https://news.asu.edu/saf/health-and-medicine,medicine
2,https://news.asu.edu/saf/local-national-and-gl...,medicine
3,https://news.asu.edu/20251202-local-national-a...,medicine
4,https://news.asu.edu/saf/arts-humanities-and-e...,medicine


Grabbing the ASU News Homepage

Now this is the most intense cell that will follow the links, avoid hitting the same URL more than once, skip duplicates and only look for news.asu.edu sites. From their it will fetch the articles and search for the keywords I set in the previous cell. If found it will then save the full URL, title, and the found keyword/s. Then I set the loop to sleep for 10 seconds to be polite to the server. This cell ended up taking over 10 minutes to run. And I ran into an issue with the titles no appearing for all the articles so I tried adding more conditions to the loop. And not able to get it to work so I adjusted the csv file and removed the title so it does not appear.



In [ ]:
homepage = requests.get("https://news.asu.edu/")
soup = BeautifulSoup(homepage.text, "html.parser")
print("ASU News")

links = soup.find_all("a")
visited = set()
for link in links:
    href = link.get("href")
    if not href:
        continue
    if href.startswith("/"):
        full_url = "https://news.asu.edu" + href
    elif href.startswith("http"):
        full_url = href
    else:
        continue
    if not full_url.startswith("https://news.asu.edu"):
        continue
    if full_url in visited:
        continue
    visited.add(full_url)
    print("Check", full_url)
    try:
        article_text = grabarticle(full_url).lower()
    except Exception as e:
        print("error", e)
        continue
    found_keywords = [kw for kw in keywords if kw.lower() in article_text]
    if found_keywords:
        print("found:", found_keywords)
        rows = [{"url": full_url, "keyword": kw} for kw in found_keywords]
        df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)
    sleep(10)
df.to_csv(csv_name, index=False)
df.tail()


ASU News
Check https://news.asu.edu/20251125-health-and-medicine-surprising-connection-between-obesity-choline-and-brain-inflammation
found: ['medicine']
Check https://news.asu.edu/saf/health-and-medicine
found: ['medicine']
Check https://news.asu.edu/saf/local-national-and-global-affairs
found: ['medicine']
Check https://news.asu.edu/20251202-local-national-and-global-affairs-college-dropout-asu-alum-space-force-general
found: ['medicine']
Check https://news.asu.edu/saf/arts-humanities-and-education
found: ['medicine']
Check https://news.asu.edu/20251121-arts-humanities-and-education-asu-again-among-top-contributors-teach-america
found: ['medicine']
Check https://news.asu.edu/quicktabs/nojs/arizona_s_shared_priorities/0
found: ['medicine']
Check https://news.asu.edu/quicktabs/nojs/arizona_s_shared_priorities/1
found: ['medicine']
Check https://news.asu.edu/quicktabs/nojs/arizona_s_shared_priorities/2
found: ['medicine']
Check https://news.asu.edu/quicktabs/nojs/arizona_s_shared_priori

,url,keyword
72,https://news.asu.edu/20251204-sun-devil-commun...,medicine
73,https://news.asu.edu/20251204-sun-devil-commun...,medicine
74,https://news.asu.edu/content/september-2025-0,medicine
75,https://news.asu.edu/thrive-magazine,medicine
76,https://news.asu.edu/contact,medicine
